In [1]:
# Import Semua Fungsi
import tensorflow as tf
import pandas as pd
import zipfile, os, shutil
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm as tq
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

In [2]:
# Cek Versi Tensorflow
print(tf.__version__)

2.5.0


In [3]:
# Download Dataset Katarak
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download jr2ngb/cataractdataset
! chmod 600 kaggle.json
! ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle
! mv kaggle.json ~/.kaggle
! kaggle datasets download jr2ngb/cataractdataset

100% 3.34G/3.34G [00:38<00:00, 49.8MB/s]
100% 3.34G/3.34G [00:38<00:00, 92.8MB/s]
cataractdataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
# Proses Ekstrak Dataset
with zipfile.ZipFile('cataractdataset.zip','r') as z:
  z.extractall('./')

os.listdir()

['.config',
 'repository',
 'cataractdataset.zip',
 'README.md',
 'dataset',
 'sample_data']

In [5]:
# Hapus Direktori yang Tidak Digunakan
dir_path = '/content/repository'

shutil.rmtree(dir_path)

In [6]:
base_dir = '/content/dataset'

file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(base_dir):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [7]:
# Menerapkan Fungsi Callback
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.75 and logs.get('val_accuracy')>0.75):
            print("\nAkurasi telah mencapai lebih dari 75%, proses dihentikan!")
            self.model.stop_training = True

In [8]:
# Proses Split Dataset
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()

X = df['path']
y = df['tag']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=300)

df_tr = pd.DataFrame({'path':X_train, 'tag':y_train, 'set':'train'})
df_val = pd.DataFrame({'path':X_val, 'tag':y_val, 'set':'validation'})

print('train size', len(df_tr))
print('val size', len(df_val))

df_all = df_tr.append([df_val]).reset_index(drop=1)
print(df_all.groupby(['set','tag']).size(),'\n')

df_all.sample(3)

train size 540
val size 61
set         tag             
train       1_normal            268
            2_cataract           92
            2_glaucoma           89
            3_retina_disease     91
validation  1_normal             32
            2_cataract            8
            2_glaucoma           12
            3_retina_disease      9
dtype: int64 



,path,tag,set
532,/content/dataset/3_retina_disease/Retina_060.png,3_retina_disease,train
568,/content/dataset/1_normal/NL_166.png,1_normal,validation
87,/content/dataset/1_normal/NL_081.png,1_normal,train


In [9]:
datasource_path = base_dir
dataset_path = "content/dataset/dataset/"

for index, row in tq(df_all.iterrows()):
    
    # Deteksi path
    file_path = row['path']            
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])  
    
    # Membuat direktori tujuan
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    # Mendefinisikan variable dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    # copy file dari direktori source ke direktori dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

In [10]:
# Image Augmentation
train_dir = "content/dataset/dataset/train"
val_dir = "content/dataset/dataset/validation"

train_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=40,
                  vertical_flip=True,
                  shear_range = 0.2,
                  zoom_range=0.2,
                  fill_mode = 'nearest')

val_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=40,
                  vertical_flip=True,
                  shear_range = 0.2,
                  zoom_range=0.2,
                  fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150,150),          
        class_mode='categorical')
 
validation_generator = val_datagen.flow_from_directory(
        val_dir, # direktori data validasi
        target_size=(150, 150), 
        class_mode='categorical')

Found 540 images belonging to 4 classes.
Found 61 images belonging to 4 classes.


In [11]:
base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

87916544/87910968 [==============================] - 1s 0us/step


In [12]:
for layer in base_model.layers:
    layer.trainable = False

In [13]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(4, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
history = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 15, epochs = 30)

Epoch 1/30
15/15 [==============================] - 77s 5s/step - loss: 2.7211 - accuracy: 0.4055 - val_loss: 1.6717 - val_accuracy: 0.4918
Epoch 2/30
15/15 [==============================] - 73s 5s/step - loss: 1.8044 - accuracy: 0.5189 - val_loss: 1.6251 - val_accuracy: 0.5574
Epoch 3/30
15/15 [==============================] - 74s 5s/step - loss: 2.2846 - accuracy: 0.4771 - val_loss: 1.5270 - val_accuracy: 0.4918
Epoch 4/30
15/15 [==============================] - 73s 5s/step - loss: 2.0445 - accuracy: 0.4454 - val_loss: 1.2347 - val_accuracy: 0.6066
Epoch 5/30
15/15 [==============================] - 73s 5s/step - loss: 1.7514 - accuracy: 0.4685 - val_loss: 1.0914 - val_accuracy: 0.6230
Epoch 6/30
15/15 [==============================] - 74s 5s/step - loss: 1.3794 - accuracy: 0.5210 - val_loss: 1.2110 - val_accuracy: 0.6066
Epoch 7/30
15/15 [==============================] - 74s 5s/step - loss: 1.3768 - accuracy: 0.5208 - val_loss: 1.0716 - val_accuracy: 0.5410
Epoch 8/30
15/15 [==